In [8]:
import pandas as pd
import numpy as np 
import requests
from bs4 import BeautifulSoup as bs

print("Libraries imported")

Libraries imported


# Assignment 1

In [9]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [10]:
source = requests.get(wiki_url).text
tront_xml = bs(source, 'lxml')
table = tront_xml.find('table')

In [13]:
column_names = ['Postal Code', 'Borough', 'Neighborhood']
df_tront = pd.DataFrame(columns = column_names)
df_tront.head()

,Postal Code,Borough,Neighborhood


In [14]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df_tront.loc[len(df_tront)] = row_data
df_tront.head()
    

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
df_boroughs = df_tront[df_tront.Borough !="Not assigned"].reset_index()
df_boroughs = df_boroughs.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
(df_boroughs['Neighborhood']=="Not assigned").any()
df_boroughs.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
df_boroughs.shape

(103, 3)

# Assignment 2

In [17]:
geo_url='https://cocl.us/Geospatial_data/Geospatial_Coordinates,csv'

In [18]:
df_geo=pd.read_csv(geo_url)
df_geo.head

<bound method NDFrame.head of     Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750072 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24          M2R  43.

In [19]:
df_hoods = pd.merge(df_boroughs, df_geo, on = 'Postal Code')
df_hoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Assignment 3

In [26]:
import json
!pip install geopy
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium

print ('Libraries imported.')


     |████████████████████████████████| 94 kB 5.7 MB/s  eta 0:00:01
Libraries imported.


In [27]:
import folium

In [28]:
address = 'Tront'
geolocator = Nominatim(user_agent="tront_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tront are{}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tront are53.87992745, 142.8679673023288.


In [31]:
map_tront = folium.Map(location=[latitude, longitude], zoom_start=10)

map_tront

In [32]:
CLIENT_ID = 'DIBJ2ZVWKDULZQLH33LHHNRZKT4W1FATCORHRVYU1T5KTWQN'
CLIENT_SECRET = '0ZUMMLCNFV2G34NB0RJCWG1RBSJD0XEWUXFJNDVAFTDLTTBZ'
VERSION = '20190605'
LIMIT = 100

In [36]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(df_hoods['Latitude'], df_hoods['Longitude'], df_hoods['Borough'], df_hoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [40]:
borough_names = list(df_tront.Borough.unique())

boros_with_downtown = []

for x in borough_names:
    if "downtown" in x.lower():
        boros_with_downtown.append(x)
        
print(boros_with_downtown)


df_downtown = df_tront[df_tront['Borough'].isin(boros_with_downtown)].reset_index(drop=True)


df_downtown = pd.merge(df_downtown, df_geo, on = 'Postal Code')
df_downtown.head()

['Downtown Toronto']


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [41]:
downtown_data = df_tront[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

NameError: name 'df' is not defined

In [42]:

map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Borough'], df_downtown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
        names,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue'
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    
    return(nearby_venues)
    

In [44]:
downtown_venues =getNearbyVenues(names=df_downtown['Neighborhood'],
                                latitudes=df_downtown['Latitude'],
                                longitudes=df_downtown['Longitude']
                                )
print(downtown_venues.shape)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements

In [45]:
dwontwon_venues.head()

NameError: name 'dwontwon_venues' is not defined

In [46]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

NameError: name 'downtown_venues' is not defined

In [47]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix=""prefix_sep="")

downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood']

fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

SyntaxError: invalid syntax (<ipython-input-47-de2c75510828>, line 1)

In [48]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

NameError: name 'downtown_onehot' is not defined

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arrange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue',format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arrange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

AttributeError: module 'numpy' has no attribute 'arrange'

In [50]:
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

kmeans.labels_[0:10]

NameError: name 'downtown_grouped' is not defined

In [51]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df_downtown

downtown_merged = downtown_merged.join(neigborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()

NameError: name 'neighborhoods_venues_sorted' is not defined

In [52]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

x = np.arrange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1 len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers=colors = []
for lat, lon, pot, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(pot) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill=True,
        fill_color = rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

SyntaxError: invalid syntax (<ipython-input-52-127414b5e446>, line 5)